# Part X — Advanced Topics  
## 43. Internationalization (i18n) and Localization (l10n) — Multilingual Django + Correct Time Zones

Internationalization and localization are where “it works on my machine” turns into
“it works for real users”:

- UI text in multiple languages (i18n)
- correct formatting of dates, numbers, currency (l10n)
- correct time zone handling (users in different regions)
- translation workflows that don’t break your team (makemessages/compilemessages)
- URL and middleware behavior that selects language properly
- testing so you don’t ship untranslated strings or broken locale logic

This chapter is written to match what professional Django teams do.

---

## 43.0 Learning Outcomes

By the end, you should be able to:

1. Explain i18n vs l10n.
2. Configure Django for multiple languages correctly:
   - `USE_I18N`, `LANGUAGE_CODE`, `LANGUAGES`, `LOCALE_PATHS`
   - `LocaleMiddleware` order
3. Mark text for translation in:
   - templates (`{% trans %}`, `{% blocktrans %}`)
   - Python code (`gettext`, `gettext_lazy`)
   - model field labels and choices
4. Generate and compile translation files:
   - `makemessages`, `compilemessages`
5. Provide a language switcher (cookie/session-based) using Django’s built-ins.
6. Handle pluralization and variable interpolation correctly.
7. Understand and configure time zones properly:
   - `USE_TZ`, `TIME_ZONE`
   - converting to local time for display
   - storing user/org time zones and activating them per request
8. Write tests that confirm:
   - translations render in a chosen language
   - date formatting and timezone display are correct

---

## 43.1 Key Concepts (Clear Definitions)

### 43.1.1 i18n (Internationalization)
Preparing your app to support multiple languages **without rewriting it**:
- marking strings as translatable
- designing templates and UI so text can change length
- handling plural forms

### 43.1.2 l10n (Localization)
Adapting the app to a specific locale:
- date/time formats
- number formats
- currency formats (often app-specific)
- address formats (often complex)

### 43.1.3 Time zones are part of localization
A localized site that shows “wrong times” is a broken site.

---

## 43.2 Django’s i18n Stack (What Pieces Do What)

Django’s internationalization relies on:

- translation functions in Python (`gettext`, `gettext_lazy`)
- template tags (`trans`, `blocktrans`)
- middleware:
  - `LocaleMiddleware` chooses active language for the request
- a translation file workflow:
  - `.po` files (editable)
  - compiled `.mo` files (used at runtime)

---

## 43.3 Configure Languages and Locale Paths

Open your settings (preferably `config/settings/base.py` if you split settings).

### 43.3.1 Ensure i18n is enabled

```python
USE_I18N = True
```

In modern Django versions, localization formatting is effectively always enabled;
older `USE_L10N` is removed in Django 4+ (so you don’t need it).

### 43.3.2 Choose default language and timezone
Example:

```python
LANGUAGE_CODE = "en"
TIME_ZONE = "UTC"
USE_TZ = True
```

**Why UTC?**
- Store times in UTC in the database (recommended)
- Convert to user’s local time for display

### 43.3.3 Declare supported languages
Add:

```python
from django.utils.translation import gettext_lazy as _

LANGUAGES = [
    ("en", _("English")),
    ("es", _("Spanish")),
    ("ne", _("Nepali")),
]
```

#### Why use `_()` around language names?
So the language names can themselves be translated in language selection UI.

### 43.3.4 Add `LOCALE_PATHS`
Create a top-level folder:

```text
locale/
```

Then in settings:

```python
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent.parent.parent  # if using settings package
# If your BASE_DIR already points to repo root, keep it consistent.

LOCALE_PATHS = [
    BASE_DIR / "locale",
]
```

**Why LOCALE_PATHS?**
- This is where project-wide translations can live (base templates, shared UI)
- App-specific translations can also live under each app’s `locale/` directory, but
  a project-level locale is very common and easier to manage early.

---

## 43.4 Enable LocaleMiddleware (Language Selection Per Request)

In `MIDDLEWARE`, ensure `LocaleMiddleware` is present and correctly ordered.

A correct typical order is:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",

    # Locale must come after SessionMiddleware (so it can use session/cookie)
    "django.middleware.locale.LocaleMiddleware",

    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

### Why ordering matters
- `LocaleMiddleware` reads language preference from:
  - URL prefix (if you use i18n_patterns)
  - cookies (`django_language`)
  - session
  - Accept-Language header
- It needs session/cookie support, so it must run after SessionMiddleware.

---

## 43.5 Mark Strings for Translation in Templates

### 43.5.1 `{% trans %}` for simple strings

In a template:

```django
{% load i18n %}

<h1>{% trans "Articles" %}</h1>
```

Explanation:
- `load i18n` enables translation tags.
- Django extracts `"Articles"` into `.po` files.

### 43.5.2 `{% blocktrans %}` for variables and pluralization

Example with variables:

```django
{% load i18n %}

{% blocktrans with username=user.username %}
  Welcome, {{ username }}.
{% endblocktrans %}
```

Why `blocktrans`?
- It treats the whole sentence as one translatable unit.
- Languages reorder words differently; translating word-by-word often breaks grammar.

### 43.5.3 Pluralization in templates
Example: “1 comment” vs “2 comments”:

```django
{% load i18n %}

{% blocktrans count count=comments|length %}
  {{ count }} comment
{% plural %}
  {{ count }} comments
{% endblocktrans %}
```

Django will store plural forms in the `.po` file according to each language’s rules.

---

## 43.6 Mark Strings for Translation in Python Code

### 43.6.1 `gettext` for immediate translation
Use when you need translation now (runtime string based on active language).

```python
from django.utils.translation import gettext as _

message = _("Task status updated.")
```

### 43.6.2 `gettext_lazy` for “translate later”
Use for:
- model field verbose names
- form labels
- choices labels
Because those are evaluated at import time, before request language is set.

Example in a model:

```python
from django.db import models
from django.utils.translation import gettext_lazy as _

class Tag(models.Model):
    name = models.CharField(_("name"), max_length=50, unique=True)
    slug = models.SlugField(_("slug"), max_length=60, unique=True)
```

### 43.6.3 Translating choices labels (professional pattern)
In `Task.Status`:

```python
from django.db import models
from django.utils.translation import gettext_lazy as _

class Task(models.Model):
    class Status(models.TextChoices):
        OPEN = "open", _("Open")
        IN_PROGRESS = "in_progress", _("In progress")
        DONE = "done", _("Done")
        CANCELED = "canceled", _("Canceled")
```

Now admin and forms show translated labels automatically.

---

## 43.7 Translation Workflow: `.po` and `.mo`

### 43.7.1 Install gettext tools (required for compilemessages)
On Ubuntu/Debian:

```bash
sudo apt install gettext
```

On macOS (Homebrew):

```bash
brew install gettext
brew link --force gettext
```

On Windows:
- easiest approach is WSL, or install gettext binaries (varies).
- many teams standardize on Docker/WSL for translation tooling.

### 43.7.2 Extract messages: `makemessages`

From project root:

```bash
python manage.py makemessages -l es
python manage.py makemessages -l ne
```

What it does:
- scans templates and Python for marked strings
- writes translation catalogs in:
  - `locale/es/LC_MESSAGES/django.po`
  - `locale/ne/LC_MESSAGES/django.po`

### 43.7.3 Edit `.po` files
Open `locale/es/LC_MESSAGES/django.po` and you’ll see entries like:

```po
msgid "Articles"
msgstr ""
```

Fill:

```po
msgid "Articles"
msgstr "Artículos"
```

Similarly for Nepali:

```po
msgid "Articles"
msgstr "लेखहरू"
```

### 43.7.4 Compile messages: `compilemessages`

```bash
python manage.py compilemessages
```

This produces `.mo` compiled files used at runtime.

**Important:** If you change `.po` files, you must re-run `compilemessages`.

---

## 43.8 Language Switching (User Chooses a Language)

Django provides a built-in endpoint to set language.

### 43.8.1 Enable the i18n URL patterns
In `config/urls.py`, include i18n routes:

```python
from django.urls import include, path

urlpatterns = [
    # ...
    path("i18n/", include("django.conf.urls.i18n")),
]
```

This provides `POST /i18n/setlang/`.

### 43.8.2 Add a language switcher form (base template)
In `templates/partials/_nav.html` (or base), add:

```django
{% load i18n %}

<form action="{% url 'set_language' %}" method="post" style="display:inline">
  {% csrf_token %}
  <input type="hidden" name="next" value="{{ request.path }}" />
  <select name="language">
    {% get_current_language as LANGUAGE_CODE %}
    {% get_available_languages as LANGUAGES %}
    {% for lang_code, lang_name in LANGUAGES %}
      <option value="{{ lang_code }}" {% if lang_code == LANGUAGE_CODE %}selected{% endif %}>
        {{ lang_name }}
      </option>
    {% endfor %}
  </select>
  <button type="submit">{% trans "Change" %}</button>
</form>
```

#### What happens when submitted
- Django sets a cookie `django_language=<code>`
- subsequent requests use that language (via LocaleMiddleware)
- the user stays on the same page (`next` field)

### 43.8.3 Where language is stored
By default, `set_language` stores it in a cookie.
You can also store in session with settings; cookie-based is common.

---

## 43.9 URL Language Prefixes (Optional, but Common for Public Sites)

Some sites prefer URLs like:
- `/en/articles/`
- `/es/articles/`

This is helpful for:
- SEO
- explicit language switching
- links that preserve language

Django supports this via `i18n_patterns`.

### 43.9.1 Apply `i18n_patterns` in `config/urls.py`
Example:

```python
from django.conf.urls.i18n import i18n_patterns
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("i18n/", include("django.conf.urls.i18n")),
]

urlpatterns += i18n_patterns(
    path("admin/", admin.site.urls),
    path("accounts/", include("django.contrib.auth.urls")),
    path("articles/", include("articles.urls")),
    path("orgs/", include("orgs.urls")),
    path("", include("pages.urls")),
)
```

#### Important consequence
All your routes now get language prefixes:
- `/en/articles/`
- `/es/articles/`

Make sure:
- reverse() and `{% url %}` are used everywhere (no hardcoded links)
- your tests account for prefixed URLs if you enable this

**Industry advice:**  
Language-prefixed URLs are great for public content sites; for internal apps, cookie
language switching is often enough.

---

## 43.10 Time Zones: Store in UTC, Display in User/Org Time Zone

### 43.10.1 The correct baseline settings

```python
USE_TZ = True
TIME_ZONE = "UTC"
```

This means:
- Django stores datetimes in UTC in DB
- Django returns “aware” datetimes
- templates can display in local time if activated

### 43.10.2 Activating a user/org timezone per request

#### Step 1: Decide where timezone preference lives
Common places:
- user profile model (`UserProfile.timezone`)
- membership preference per org (`Membership.timezone`)
- org default (`Organization.timezone`)

For your org-scoped app, a practical pattern:
- add `Organization.timezone` as default
- allow user override later

#### Step 2: Add timezone field to Organization
In `orgs/models.py`:

```python
from django.db import models
from django.utils.translation import gettext_lazy as _

class Organization(models.Model):
    # ...
    timezone = models.CharField(
        _("timezone"),
        max_length=64,
        default="UTC",
        help_text=_("IANA timezone, e.g. 'Asia/Kathmandu' or 'Europe/Berlin'."),
    )
```

Migrate:
```bash
python manage.py makemigrations
python manage.py migrate
```

#### Step 3: Middleware to activate timezone
Create `orgs/timezone_middleware.py`:

```python
from __future__ import annotations

from django.utils import timezone


class TenantTimezoneMiddleware:
    """
    Activates timezone based on request.tenant.timezone when tenant exists.
    Falls back to default TIME_ZONE if no tenant.
    """

    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        tz = None
        tenant = getattr(request, "tenant", None)
        if tenant is not None:
            tz = getattr(tenant, "timezone", None)

        if tz:
            timezone.activate(tz)
        else:
            timezone.deactivate()

        return self.get_response(request)
```

Register it **after** TenantMiddleware (because it needs `request.tenant`) and
before views:

```python
MIDDLEWARE += [
    "orgs.timezone_middleware.TenantTimezoneMiddleware",
]
```

(Insert in correct position; keep middleware order intentional.)

### 43.10.3 Displaying times correctly in templates
In templates, use:

```django
{{ task.created_at|date:"Y-m-d H:i" }}
```

Django will format `created_at` in the active timezone (if activated).

You can also use the `localtime` filter:

```django
{% load tz %}
{{ task.created_at|localtime }}
```

### 43.10.4 Why you should never store naive datetimes
- naive datetimes have no timezone info
- they lead to “off by hours” bugs, especially around DST
- Django with `USE_TZ=True` expects aware datetimes in many places

---

## 43.11 Translation of Dates/Numbers (Localization Formatting)

Django provides template filters:

- `date`
- `time`
- `timesince`
- `localize` (format according to locale)
- `floatformat`

Example localized date:

```django
{% load l10n %}
{{ article.published_at|localize }}
```

For numbers:

```django
{% load l10n %}
{{ some_number|localize }}
```

If you build currency formatting, you often implement a custom filter that:
- uses Decimal
- uses locale formatting rules
- applies currency symbol rules
(That’s beyond basic Django; many apps use specialized libraries.)

---

## 43.12 Common Pitfalls (and How to Avoid Them)

### Pitfall A: Forgetting to load `i18n` in templates
If `{% trans %}` doesn’t work:
- ensure `{% load i18n %}` is in that template (or base template).

### Pitfall B: Strings not extracted by makemessages
- only strings wrapped in translation functions/tags are extracted
- dynamic strings like `_("Hello " + name)` are not extractable correctly

Correct pattern:
- translate the template string, not concatenated parts:

```python
_("Hello %(name)s") % {"name": name}
```

### Pitfall C: Using `gettext` instead of `gettext_lazy` for model field labels
- model fields are defined at import time
- use lazy translation for labels and choices

### Pitfall D: Not compiling messages in deployment
If translations work locally but not in production:
- `.po` edited but `.mo` not compiled
- ensure `compilemessages` is part of build/release pipeline

### Pitfall E: Timezone activated but values still look wrong
Common causes:
- stored datetimes are naive
- USE_TZ misconfigured across environments
- OS timezone and Django timezone confusion
- comparing naive and aware datetimes in code

Fix:
- keep USE_TZ consistent
- use `timezone.now()`, not `datetime.now()`
- use `timezone.localtime()` for display in Python if needed

---

## 43.13 Testing i18n and Time Zones (Don’t Ship Broken Locale Behavior)

### 43.13.1 Test a translated string in a template
Example using Django TestCase + `override`:

```python
from django.test import TestCase
from django.urls import reverse
from django.utils.translation import override


class I18nTests(TestCase):
    def test_articles_title_in_spanish(self):
        with override("es"):
            response = self.client.get(reverse("articles:list"))
            self.assertEqual(response.status_code, 200)
            # Adjust this assertion to a string you actually translated.
            self.assertContains(response, "Artículos")
```

### 43.13.2 Test timezone activation for tenant pages
Assuming TenantTimezoneMiddleware activates org timezone:

```python
from django.test import TestCase
from django.utils import timezone
from django.contrib.auth import get_user_model
from django.urls import reverse

from orgs.models import Membership, Organization
from tasks.models import Task


class TimezoneTests(TestCase):
    def test_org_timezone_affects_display(self):
        User = get_user_model()
        user = User.objects.create_user(username="u1", password="pass12345")

        org = Organization.objects.create(
            name="Acme",
            slug="acme",
            timezone="Asia/Kathmandu",
        )
        Membership.objects.create(
            organization=org,
            user=user,
            role=Membership.Role.ADMIN,
        )

        task = Task.objects.create(
            organization=org,
            title="T",
            description="x",
            status=Task.Status.OPEN,
            priority=Task.Priority.MEDIUM,
            created_by=user,
            updated_by=user,
        )

        self.client.login(username="u1", password="pass12345")
        url = reverse("tasks:detail", kwargs={"org_slug": "acme", "task_id": task.id})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)

        # We won't assert exact time text (formats vary); instead ensure page loads and shows some date label.
        self.assertContains(response, "Created")
```

For strict assertions, you can:
- freeze time
- format expected localtime string with `timezone.localtime(...)`
and compare.

---

# 43.14 Hands-On Lab (Recommended): Translate Your UI + Activate Org Time Zones

## Lab A — Add translations for your nav and key pages
1. Add `{% load i18n %}` to base or nav templates.
2. Mark strings:
   - “Home”
   - “Articles”
   - “New Article”
   - “Tasks”
   - “Login/Logout”
3. Run:
   - `makemessages -l es`
   - translate 10+ strings
   - `compilemessages`
4. Switch language using the language form and confirm UI changes.

## Lab B — Correct timezone display per org
1. Add `Organization.timezone`
2. Add TenantTimezoneMiddleware
3. Set org timezone to `Asia/Kathmandu`
4. Create a task and view created_at display
5. Confirm it changes if you switch org timezone

---

## 43.15 Exercises (Do These Before Proceeding)

1. Add translations for form errors and messages:
   - wrap `messages.success(...)` strings with `_()`
   - translate them in `.po`

2. Translate model choice labels:
   - Task status and priority labels
   - Article status labels

3. Add `LANGUAGE_COOKIE_NAME = "django_language"` explicitly (optional) and document it.

4. Enable language-prefixed URLs with `i18n_patterns` (optional):
   - update navigation links (must use `{% url %}`)
   - update one test to use correct prefixed URL or use `override` + `reverse`

5. Add a “user language preference” model:
   - store preferred language code
   - middleware: if user logged in, override cookie choice (policy decision)
   - document the precedence order you choose

---

## 43.16 Chapter Summary

- i18n: translate UI text; l10n: format dates/numbers; time zones are part of l10n.
- Use `LocaleMiddleware` and correct settings (`LANGUAGES`, `LOCALE_PATHS`).
- Mark strings properly:
  - templates: `{% trans %}`, `{% blocktrans %}`
  - Python: `gettext` / `gettext_lazy`
- Workflow: `makemessages` → edit `.po` → `compilemessages`.
- Provide a language switcher using Django’s built-in `set_language`.
- Store datetimes in UTC (`USE_TZ=True`) and activate tenant/user timezone for display.
- Test translations and timezone behavior so it doesn’t regress.

---

Next chapter: **44. Accessibility and UX Quality (Professional Baseline)**  
We’ll make your forms and templates accessible (labels, error summaries, focus
management), improve UX feedback patterns, and adopt an audit checklist that matches
industry expectations for production web apps.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='42. advanced_authorization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='44. accessibility_and_ux_quality.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
